In [1]:
import torch 
from torch import nn
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,TensorDataset

c:\Users\Avijit\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data=pd.read_csv(r"C:\Users\Avijit\Desktop\AgriAssure\AGRIYIELD PRO\PROCESSED_DATASET\yield.csv")

In [3]:
x1=data.iloc[:,:-1]
y1=data.iloc[:,-1:]

In [4]:
yscaler=StandardScaler()


In [5]:
Yscale=yscaler.fit_transform(y1)

In [6]:
x=torch.tensor(x1.values,dtype=torch.float)
y=torch.tensor(Yscale,dtype=torch.float)

In [40]:
class yieldpredicionModelv1(nn.Module):
    def __init__(self,input_shape:int,hidden_units:int,output_shape:int ):
        super().__init__()
        self.regressor=nn.Sequential(
            nn.Linear(in_features=input_shape,out_features=hidden_units),
            nn.PReLU(),
            nn.Dropout(0.3),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.PReLU(),
            nn.Dropout(0.1),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.PReLU(),
            nn.Linear(in_features=hidden_units,out_features=output_shape)
        )
    def forward(self,x):
        return self.regressor(x)

In [41]:
torch.manual_seed(42)
# model=yieldpredicionModelv1(input_shape=5,hidden_units=128,output_shape=1).to(device='cuda')
device='cuda' if torch.cuda.is_available() else 'cpu'
model=yieldpredicionModelv1(input_shape=x.shape[1],hidden_units=128,output_shape=1).to(device)

In [42]:
loss_fn=torch.nn.SmoothL1Loss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.5)


In [43]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)

In [44]:
device='cuda'

In [45]:
def train(model, xtrain, ytrain, lossfn, optimizer, device):
    model.train()
    xtrain = xtrain.to(device)
    ytrain = ytrain.to(device)

    trainpred = model(xtrain)
    loss = lossfn(trainpred, ytrain)
    r2 = r2_score(ytrain.detach().cpu().numpy(), trainpred.detach().cpu().numpy())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"TRAINLOSS: [{loss.item()}] TRAIN-R2SCORE: [{r2}]")
    return loss.item(), r2


def evaluate_model(model, xtest, ytest, loss_fn, device):
    model.eval()
    xtest, ytest = xtest.to(device), ytest.to(device)

    with torch.inference_mode():
        pred = model(xtest)
        loss = loss_fn(pred, ytest)
        
        y_true = ytest.detach().cpu().numpy()
        y_pred = pred.detach().cpu().numpy()
        r2 = r2_score(y_true, y_pred)
    print(f"TESTLOSS: [{loss.item()}] TEST-R2SCORE: [{r2}]")
    return loss.item(), r2


In [46]:
epochs=10000
patience = 1000
best_loss = float('inf')
counter = 0
best_model_state = None
early_stop = False
train_loss,train_r2,test_loss,test_r2=[],[],[],[]

for epoch in range(epochs):
    print(f"EPOCH-->{epoch}")
    trainloss,trainr2 = train(model, xtrain, ytrain, loss_fn, optimizer, device)
    train_loss.append(trainloss)
    train_r2.append(trainr2)
    testloss, testr2 = evaluate_model(model, xtest, ytest, loss_fn, device)
    test_loss.append(testloss)
    test_r2.append(testr2)

    if testloss < best_loss:
        best_loss = testloss
        best_model_state = model.state_dict()
        counter = 0 
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch}")
            early_stop = True
            break

if early_stop:
    model.load_state_dict(best_model_state)


EPOCH-->0
TRAINLOSS: [0.42748773097991943] TRAIN-R2SCORE: [0.0031471848487854004]
TESTLOSS: [0.42427271604537964] TEST-R2SCORE: [0.011024296283721924]
EPOCH-->1
TRAINLOSS: [0.41865888237953186] TRAIN-R2SCORE: [0.01178830862045288]
TESTLOSS: [0.4190003573894501] TEST-R2SCORE: [0.011478304862976074]
EPOCH-->2
TRAINLOSS: [0.4141194820404053] TRAIN-R2SCORE: [0.007093608379364014]
TESTLOSS: [0.41572171449661255] TEST-R2SCORE: [0.010202527046203613]
EPOCH-->3
TRAINLOSS: [0.4092363715171814] TRAIN-R2SCORE: [0.011549770832061768]
TESTLOSS: [0.41381242871284485] TEST-R2SCORE: [0.009496748447418213]
EPOCH-->4
TRAINLOSS: [0.40746545791625977] TRAIN-R2SCORE: [0.004491031169891357]
TESTLOSS: [0.41296449303627014] TEST-R2SCORE: [0.009673535823822021]
EPOCH-->5
TRAINLOSS: [0.4061107635498047] TRAIN-R2SCORE: [0.006627023220062256]
TESTLOSS: [0.4130610227584839] TEST-R2SCORE: [0.0100516676902771]
EPOCH-->6
TRAINLOSS: [0.4056314527988434] TRAIN-R2SCORE: [0.005061089992523193]
TESTLOSS: [0.41345450282096